In [1]:
import numpy as np
import pandas as pd

In [2]:
# Import phi from train data set
phi = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
                 usecols=tuple(range(1, 14)))

# Import phi_test from test data set
phi_test = np.loadtxt('test.csv', dtype='float', delimiter=',',
                      skiprows=1, usecols=tuple(range(1, 14)))

# Import y from train data set
y = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
               usecols=14, ndmin=2)

In [3]:
# Length of datasets
print("length of training dataset")
phi_len=len(phi)
print(phi_len)
print("length of testing dataset")
phi_test_len=len(phi_test)
print(phi_test_len)

length of training dataset
400
length of testing dataset
105


In [4]:
# Concatenate coloumn of 1s to right of phi and phi_test
phi_test = np.concatenate((phi_test, np.ones((phi_test_len, 1))), axis=1)
phi = np.concatenate((phi, np.ones((phi_len, 1))), axis=1)

In [5]:
# min max scaling on each coloumn of phi and phi_test
for i in range(0, 13):
    each_col_max = max(phi[:, i])
    each_col_min = min(phi[:, i])
    phi[:, i] = (phi[:, i] - each_col_min) / (each_col_max - each_col_min)
    phi_test[:, i] = (phi_test[:, i] - each_col_min) / (each_col_max - each_col_min)

In [6]:
# Log scaling on y
y = np.log(y)

In [7]:
# calculation of change in error function
def delta(p, phi, w):
    if p == 2:
        delta_item = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambda_value * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        delta_item = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambda_value * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return delta_item

In [8]:
# Dictionary containing filenames as keys and p as values
filenames = {'Dictionary_out.csv': 2.0,
             'Dictionary_out_p1.csv': 1.75,
             'Dictionary_out_p2.csv': 1.5,
             'Dictionary_out_p3.csv': 1.3
             }

In [9]:
# Operation for each item in this dictionary
for (f_item, p) in filenames.items():
    
    w = np.zeros((14, 1))  # Setting of initial w to zeros
    lambda_value = 0.2            # Hyperparameter lambda value
    t = 0.00012            # Maximum step size

    # Calculation of  new value of w
    w_new = w - t * delta(p, phi, w)

    i = 0
    
    # Repeat steps until error between consecutive w is less than threshold
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * delta(p, phi, w)
        i = i + 1

    # Load values of id
    id_test = np.loadtxt('data/test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)

    # Calculate y for test data
    y_test = np.exp(np.dot(phi_test, w_new))

    # Save the ids and y
    np.savetxt(f_item, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')